# Goals:
## 1. Call common peaks from each sample so that there are no missing peaks in any sample when merging the data. 
## 2. Create chromatic assay object for each sample and QC

In [ ]:
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Mmusculus.v79)
library(ggplot2)
library(glue)
library(scDblFinder)
library(RColorBrewer)
library(dplyr)
library(ggridges)
library(CopyscAT)
library(BSgenome.Mmusculus.UCSC.mm10)
library(GenomicRanges)
library(future)
set.seed(123)


# <span style="color:green"> Part 1 - Call common peaks and creat ATAC object<span>

In [ ]:
## setting up folder structure 
sample_id <- "KO1"

In [ ]:
# create folders to store single sample processing outputs for each sample

system("mkdir data")
system("mkdir data/single_sample_processing")
system(glue("mkdir data/single_sample_processing/{sample_id}"))

## Filtering out "non-cells" based on CellRanger cell calling using CellRanger barcode output file

In [ ]:
barcodes <- read.table(glue('data/single_sample_processing/{sample_id}/cellranger/barcodes.tsv.gz'))

In [ ]:
head(barcodes)

In [ ]:
dim(barcodes)

In [ ]:
# filter fragment file, this creates a new fragment file w/ name specified by outfile argument
FilterCells(glue('data/single_sample_processing/{sample_id}/cellranger/atac_fragments.tsv.gz'), barcodes$V1, outfile = glue('data/single_sample_processing/{sample_id}/cellranger/fragments.filtered.tsv.gz'))


## Create matrix of peaks X cells using a set of common peaks  

### Call peaks for each sample

In [ ]:
# downloading packages 

system('pip install macs2', intern = TRUE)
system('pip install numpy==1.21.6', intern = TRUE)
system('which macs2', intern = TRUE)

In [ ]:
samples <- c("KO1", "KO2", "WT1", "WT2")

In [ ]:
for (sample_id in samples) {
    # Create the file path using glue
    fragment_file_path <- glue('data/single_sample_processing/{sample_id}/cellranger/fragments.filtered.tsv.gz')
    
    # Dynamically assign the output of CallPeaks to a variable with the name "peaks_<sample_id>"
    assign(glue("peaks_{sample_id}"), CallPeaks(
        object = fragment_file_path,
        macs2.path = '/home/jupyter/.local/bin/macs2'
    ))
}

In [ ]:
peaks_WT2


In [ ]:
peaks_KO2

## Create a unified/common set of peaks to quantify in each dataset

In [ ]:
combined_peaks  <- reduce(x = c(peaks_KO1, peaks_KO2, peaks_WT1, peaks_WT2))


In [ ]:
combined_peaks

##  Create Fragment Object for each sample 

In [ ]:
for (sample_id in samples){
    fragment_file_path = glue('data/single_sample_processing/{sample_id}/cellranger/fragments.filtered.tsv.gz')
    
    assign(glue("fragments_{sample_id}"), CreateFragmentObject(fragment_file_path))
}


## Create a matrix of peaks x cell for each sample using combined_peaks 

In [ ]:
for (sample_id in samples){
    
    # Retrieve fragment and peak objects by their names
    fragments = get(glue("fragments_{sample_id}"))
    
    assign(glue("counts_{sample_id}"), FeatureMatrix(fragments, combined_peaks))

}

In [ ]:
dim(counts_KO1)
dim(counts_KO2)
dim(counts_WT1)
dim(counts_WT2)

##  Construct chromatin assay - do this for each smaple by changing the smaple_id and fragment_file_path

In [ ]:
sample_id <- "KO1" # change this for each sample
fragment_file_path <- glue('data/single_sample_processing/{sample_id}/cellranger/fragments.filtered.tsv.gz')


In [ ]:
# all default parameters
chrom_assay = CreateChromatinAssay(
    counts = get(glue("counts_{sample_id}")),
    sep = c(":", "-"),
    genome = 'mm10', #mice
    fragments = get(glue("fragments_{sample_id}")),
    min.cells = 10,
    min.features = 200)

In [ ]:
dim(chrom_assay)

In [ ]:
obj = CreateSeuratObject(
    counts = chrom_assay,
    assay = "peaks",
    )

In [ ]:
obj

### Add annotations

In [ ]:
# extract gene annotations from EnsDb
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)

# change to UCSC style since the data was mapped to hg19
seqlevels(annotations) <- paste0('chr', seqlevels(annotations))
genome(annotations) <- "mm10"


In [ ]:
# add the gene information to the object
Annotation(obj) <- annotations 

In [ ]:
head(seqnames(obj))

In [ ]:
saveRDS(obj, "KO1_atac_Allcell_obj_cleaned.RDS")


In [ ]:
dim(obj)
length(colnames(obj))

# <span style="color:green"> Part 2a - QC:remove doublets. <span>
Calling doublets for each sample one by one using scDblFinder. <br>
documentation followed: https://www.bioconductor.org/packages//release/bioc/vignettes/scDblFinder/inst/doc/scDblFinder.html
#some-important-parameters section

## Call doublets

In [ ]:
system(glue('mkdir data/single_sample_processing/{sample_id}/qc'))

In [ ]:
peak_assay <- GetAssayData(object = obj, assay = "peaks", layer = "data")


In [ ]:
sce <- scDblFinder(peak_assay, aggregateFeatures=TRUE, nfeatures=25, processing="normFeatures", artificialDoublets = length(colnames(obj)))
obj$doublet_class <- sce@colData$scDblFinder.class
obj$doublet_class <- factor(obj$doublet_class, levels = c('singlet', 'doublet'))
obj$doublet_score <- sce@colData$scDblFinder.score

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)

g = ggplot(obj[[]], aes_string(x = 'doublet_score', color = 'doublet_class', fill = 'doublet_class')) + geom_histogram()
g + scale_color_manual(name = 'Final classification', values=c("#0571b0", "#ca0020"))+
  scale_fill_manual(name = 'Final classification',values=c("#0571b0", "#ca0020")) + theme_classic() + ylab('Number of cells') + xlab('Probability of cell being a\ndoublet (score)')+ 
theme(axis.ticks.x=element_blank(), axis.ticks.y=element_blank(), text = element_text(size = 16, family = 'Helvetica'), legend.title=element_text(size=14 ,family = 'Helvetica'), legend.position = 'right')
ggsave(glue('data/single_sample_processing/{sample_id}/qc/doubscores.pdf'), width = 8, height = 6)


In [ ]:
stacked_bar <- dplyr::count(obj[[]], obj[[]]$doublet_class, sort = T)
stacked_bar$type = 'tmp'
ggplot(stacked_bar, aes(fill = stacked_bar[,1], y = stacked_bar[,2], x = stacked_bar[,3])) +
geom_bar(position = 'fill', stat = 'identity', width = 0.6) + scale_color_manual(name = 'Final classification', values=c("#0571b0", "#ca0020"))+
  scale_fill_manual(name = 'Final classification',values=c("#0571b0", "#ca0020")) + theme_classic() + ylab('Proportion of cells') + xlab('')+ 
theme(aspect.ratio = 3/1, axis.ticks.x=element_blank(), axis.text.x=element_blank(), text = element_text(size = 16, family = 'Helvetica'), legend.title=element_text(size=14 ,family = 'Helvetica'), legend.position = 'right')

ggsave(glue('data/single_sample_processing/{sample_id}/qc/doubproportions.pdf'), width = 8, height = 6)

In [ ]:
dim(obj)

In [ ]:
sum(obj$doublet_class == "doublet")

## Remove doublets

In [ ]:
obj <- subset(
  x = obj,
  subset = doublet_class == 'singlet'
)
dim(obj)

In [ ]:
saveRDS(obj, "KO1_atac_ALLsinglet_obj.RDS")

# <span style="color:green"> Part 2 - QC: Compute QC metrics and visualizing distributions for each sample one by one

## Count fragments

In [ ]:
fragment_file_path # double-check if the path is correct

In [ ]:
total_fragments <- CountFragments(fragment_file_path)

## Calculate fraction of reads in peaks for each cell 
Calculate the proportion of sequencing reads (fragments) from a single cell that fall within regions identified as "peaks" in a chromatin accessibility analysis

In [ ]:
# Assign the total fragment calculated from CountFragment() function to the metadata
row.names(total_fragments) <- total_fragments$CB
obj$fragments <- total_fragments[colnames(obj), "frequency_count"]

# Use FRiP() function to calculate the fraction of reads in peaks per cell
obj <- FRiP(
    object = obj,
    assay = 'peaks',
    total.fragments = 'fragments')

## counting fragments in genome blacklist regions (can be diagnostic of low quality cells)

In [ ]:
obj$blacklist_fraction <- FractionCountsInRegion(
    object = obj,
    assay = 'peaks',
    regions = blacklist_mm10)

## compute nucleosome signal score per cell -- proxy of fragment length periodicity

In [ ]:
obj <- NucleosomeSignal(object = obj)

## compute TSS enrichment score per cell

In [ ]:
obj <- TSSEnrichment(object = obj, fast = FALSE)

## Calculate peak region fragments using colSums():
* computes the sum of values for each column of the matrix. In the case of single-cell ATAC-seq data, each column represents a cell, and each row represents a peak (a genomic region with accessible chromatin).
* therefore computes the overall chromatin accessiblity for each cell

In [ ]:
obj$peak_region_fragments <- colSums(GetAssayData(obj, assay = "peaks", layer = "data"))

In [ ]:
options(repr.plot.width=10, repr.plot.height=6)

obj$high.tss <- ifelse(obj$TSS.enrichment > 2, 'TSS enrichment > 2', 'TSS enrichment < 2')
TSSPlot(obj, group.by = 'high.tss') + NoLegend()  & scale_color_manual(values = c('black', 'black')) & 
theme(plot.title = element_blank(), text = element_text(size = 18)) 

ggsave(glue('data/single_sample_processing/{sample_id}/qc/tssenrichment.pdf'), width = 10, height = 6)

In [ ]:
options(repr.plot.width=7, repr.plot.height=6)
DensityScatter(obj, x = 'nCount_peaks', y = 'TSS.enrichment', log_x = TRUE, quantiles = TRUE)

In [ ]:
options(repr.plot.width=20, repr.plot.height=6)
titles <- c('Fraction of reads\nin peaks', 'Total # of fragments\nin peaks', 'Transcription start site\nenrichment', 'Blacklist fraction', 'Nucleosome signal')
colnames <- c('FRiP','nCount_peaks','TSS.enrichment', 'blacklist_fraction', 'nucleosome_signal')

plot_lst <- vector('list', length = 0)
for (i in 1:length(colnames)) {
    g <- ggplot(obj[[]], aes_string(x = factor(0), y = obj[[]][,colnames[i]])) + theme_classic() + geom_violin(color = 'black', fill = 'grey') +
    theme(axis.ticks.x=element_blank(), axis.ticks.y=element_blank(),axis.title.x=element_blank(),axis.text.x=element_blank(), axis.text.y=element_text(size = 15), text = 
    element_text(size = 16, family = 'Helvetica'), axis.title.y=element_blank(), legend.position = "none") + geom_boxplot(width = 0.1) +
    ggtitle(titles[i]) + theme(plot.title = element_text(hjust = 0.5, family = 'Helvetica'))
    plot_lst[[i]] <- g 
}


cowplot::plot_grid(plotlist = plot_lst, nrow= 1, align = 'h')
ggsave(glue('data/single_sample_processing/{sample_id}/qc/quality_distributions_violin.pdf'), width = 20, height = 6)

# <span style='color:green'> Remove cells that are outliers based on these QC metrics, determine thresholds dynamically (each sample have unique threshold based on the distribution) </span>

In [ ]:
## determine cut offs based on 3 MAD from median 
options(repr.plot.width=20, repr.plot.height=4)
titles <- c('Fraction of reads\nin peaks', 'Total # of fragments\nin peaks', 'Transcription start site\nenrichment', 'Blacklist fraction', 'Nucleosome signal')
colnames <- c('FRiP','nCount_peaks','TSS.enrichment', 'blacklist_fraction', 'nucleosome_signal')

plot_lst <- vector('list', length = 0)
for (i in 1:length(colnames)) {
    g <- ggplot(obj[[]], aes_string(x = colnames[i])) + geom_density(fill = 'grey', color = '#616161')
    g <- g + geom_vline(aes_string(xintercept = median(obj[[colnames[i]]][,1]) + (3 * mad(obj[[colnames[i]]][,1]))))
    g <- g + geom_vline(aes_string(xintercept = median(obj[[colnames[i]]][,1]) - (3 * mad(obj[[colnames[i]]][,1]))))
    g <- g + ggtitle(paste('mincutoff = ', round(median(obj[[colnames[i]]][,1]) - (3 * mad(obj[[colnames[i]]][,1])), 2), '\nmaxcutoff = ', round(median(obj[[colnames[i]]][,1]) + (3 * mad(obj[[colnames[i]]][,1])), 2)))
    g <- g + theme_classic() + theme(axis.ticks.x=element_blank(), axis.ticks.y=element_blank(), axis.text.y=element_text(size = 15), text = 
    element_text(size = 14, family = 'Helvetica'), legend.position = "none", plot.title = element_text(family = 'Helvetica', size = 15)) + xlab(titles[i])
    plot_lst[[i]] <- g
}


cowplot::plot_grid(plotlist = plot_lst, nrow= 1, align = 'h')
ggsave(glue('data/single_sample_processing/{sample_id}/qc/quality_distributions_cutoff.pdf'), width = 20, height = 4)

In [ ]:
colnames <- c('FRiP', 'nCount_peaks', 'TSS.enrichment', 'blacklist_fraction', 'nucleosome_signal')
cutoffs <- list()

for (i in colnames) {
    # Calculate max and min cutoffs
    maxcutoff <- round(median(obj[[i]][, 1]) + (3 * mad(obj[[i]][, 1])), 2)
    mincutoff <- round(median(obj[[i]][, 1]) - (3 * mad(obj[[i]][, 1])), 2)
    
    # Store the cutoffs in the list
    cutoffs[[i]] <- list(max = maxcutoff, min = mincutoff)
}

# View the cutoffs
print(cutoffs)


In [ ]:
obj <- subset(
    x = obj,
    subset = nCount_peaks < 30691.21 &
    nCount_peaks > 3000 & # > mincutoff
    FRiP > 0.47 & # > mincutoff
    blacklist_fraction < 0.05 & # < maxcutoff
    nucleosome_signal < 1.11 &  # < maxcutoff
    TSS.enrichment > 2.42   # > mincutoff
)
dim(obj)

In [ ]:
saveRDS(obj, "KO1_atac_QCfiltered.RDS")
